In [1]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [2]:
signal_color = ctrl.Antecedent(np.arange(0, 3, 1), 'signal_color')  # 0: Red, 1: Yellow, 2: Green
distance = ctrl.Antecedent(np.arange(0, 101, 1), 'distance')        # Distance to signal (0 to 100 meters)

In [4]:
deceleration = ctrl.Consequent(np.arange(0, 11, 1), 'deceleration')  # 0 to 10 units
stop_decision = ctrl.Consequent(np.arange(0, 2, 1), 'stop_decision')  # 0: Stop, 1: Go

In [5]:
signal_color['red'] = fuzz.trimf(signal_color.universe, [0, 0, 0])
signal_color['yellow'] = fuzz.trimf(signal_color.universe, [1, 1, 1])
signal_color['green'] = fuzz.trimf(signal_color.universe, [2, 2, 2])

# Membership functions for distance
distance['close'] = fuzz.trimf(distance.universe, [0, 0, 40])
distance['medium'] = fuzz.trimf(distance.universe, [30, 50, 70])
distance['far'] = fuzz.trimf(distance.universe, [60, 100, 100])

# Membership functions for deceleration
deceleration['none'] = fuzz.trimf(deceleration.universe, [0, 0, 2])
deceleration['moderate'] = fuzz.trimf(deceleration.universe, [3, 5, 7])
deceleration['high'] = fuzz.trimf(deceleration.universe, [7, 10, 10])

# Membership functions for stop_decision
stop_decision['stop'] = fuzz.trimf(stop_decision.universe, [0, 0, 0])
stop_decision['go'] = fuzz.trimf(stop_decision.universe, [1, 1, 1])


In [6]:
rule1 = ctrl.Rule(signal_color['red'] & distance['close'], 
                  consequent=[deceleration['high'], stop_decision['stop']])

rule2 = ctrl.Rule(signal_color['green'] & distance['far'], 
                  consequent=[deceleration['none'], stop_decision['go']])

rule3 = ctrl.Rule(signal_color['yellow'] & distance['medium'], 
                  consequent=[deceleration['moderate'], stop_decision['stop']])

In [7]:
traffic_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
traffic_simulation = ctrl.ControlSystemSimulation(traffic_ctrl)

# Example input
traffic_simulation.input['signal_color'] = 0  # Red
traffic_simulation.input['distance'] = 20     # Close
traffic_simulation.compute()

print(f"Recommended Deceleration: {traffic_simulation.output['deceleration']:.2f}")
print(f"Decision (0 = Stop, 1 = Go): {traffic_simulation.output['stop_decision']:.2f}")


Recommended Deceleration: 8.83
Decision (0 = Stop, 1 = Go): 0.39
